<a href="https://colab.research.google.com/github/JapiKredi/Mistral_Falcon_HF_Question-Answering_PDF./blob/main/Mistral_Falcon_HF_Question%26Answering_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun 17 13:55:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# install the pypdf library
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.4 MB/s eta 0:00:00


In [3]:
# Install required libraries for deep learning and NLP tasks (quiet mode)
!pip install -q transformers einops accelerate bitsandbytes langchain

# Explanation:
#   - transformers: Provides pre-trained models for various NLP tasks
#   - einops: Efficient operations for manipulating multidimensional arrays
#   - accelerate: Optimizations for deep learning training
#   - bitsandbytes: Tools for working with binary data
#   - langchain: Building blocks for NLP workflows (data, processing, retrieval, etc.)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.9 MB/s eta 0:00:00


In [4]:
# Install the sentence-transformers library
!pip install sentence-transformers

# Explanation:
#   This library provides pre-trained models for generating numerical representations (embeddings) of sentences.
#   These embeddings can be used for various NLP tasks like text classification, semantic search, and more.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.7 MB/s eta 0:00:00


In [5]:
# Install llama-index library (specific version 0.9.39)
!pip install llama-index==0.9.39

# Explanation:
#   This installs the llama-index library, a framework for building workflows
#   that involve large language models (LLMs) and vector databases.
#   Version 0.9.39 is specified for reproducibility or compatibility reasons
#   (adjust if needed).

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.3 MB/s eta 0:00:00


In [6]:
# Imports from llama_index library
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

# Import for interacting with Hugging Face Transformers models
from llama_index.llms import HuggingFaceLLM

# Import for defining simple input prompts
from llama_index.prompts.prompts import SimpleInputPrompt

In [7]:
# Create a directory named "Data" (if it doesn't exist)
!mkdir Data

In [10]:
# Load text data from the "Data" directory
documents = SimpleDirectoryReader("/content/Data").load_data()

In [11]:
documents

[Document(id_='7a1229b1-b8ff-48b3-9116-ed53c4e2f63e', embedding=None, metadata={'page_label': '1', 'file_name': 'Attention_paper.pdf', 'file_path': '/content/Data/Attention_paper.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-06-17', 'last_modified_date': '2024-06-17', 'last_accessed_date': '2024-06-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Res

In [12]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Mistral LLM model

In [14]:
# Mistral (code using Mistral LLM)

import torch

# Create a HuggingFaceLLM instance for Mistral model (mistralai/Mistral-7B-Instruct-v0.1)
llm = HuggingFaceLLM(
    context_window=4096,  # Maximum context window size (in tokens)
    max_new_tokens=256,  # Maximum number of tokens to generate per call
    generate_kwargs={  # Keyword arguments for generating text
        "temperature": 0.0,  # No randomness (greedy decoding)
        "do_sample": False,   # Don't use random sampling during generation
    },
    system_prompt=system_prompt,  # Predefined system prompt for the LLM (replace with actual prompt)
    query_wrapper_prompt=query_wrapper_prompt,  # Predefined query wrapper prompt for the LLM (replace with actual prompt)
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",  # Name of the tokenizer for the Mistral model
    model_name="mistralai/Mistral-7B-Instruct-v0.1",  # Name of the Mistral model on Hugging Face Hub
    device_map='auto',  # Automatically select device (CPU or GPU)
    # Options to reduce memory usage (uncomment if using CUDA):
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [15]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.2 MB/s eta 0:00:00


In [16]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

In [17]:
embed_model = LangchainEmbedding(
    HuggingFaceBgeEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
service_context = ServiceContext.from_defaults(
    chunk_size = 1024,
    llm = llm,
    embed_model = embed_model
)

In [19]:
index = VectorStoreIndex.from_documents(documents, service_context= service_context)

In [20]:
query_engine = index.as_query_engine()

In [21]:
#query = "What is the difference between attention and self-attention?"
query = "What types of attention are mentioned in the 'All you need is attention paper', and what is the difference between them?"

response = query_engine.query(query)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The 'All you need is attention paper' mentions two types of attention: Input-Input Layer5 and Attention Visualizations. Input-Input Layer5 is a type of attention that is used to resolve anaphora, which is the repetition of the same word or phrase at the beginning of successive clauses or sentences. Attention Visualizations, on the other hand, is a type of attention that is used to follow long-distance dependencies in the encoder self-attention in layer 5 of 6. The difference between them is that Input-Input Layer5 is used to resolve anaphora, while Attention Visualizations is used to follow long-distance dependencies in the encoder self-attention in layer 5 of 6.


# Falcon LLM Model

In [22]:
# Falcon

# import torch

llm = HuggingFaceLLM(
context_window = 4096,
max_new_tokens = 256,
generate_kwargs = {"temperature": 0.0, "do_sample": False},
system_prompt = system_prompt,
query_wrapper_prompt= query_wrapper_prompt,
tokenizer_name = "tiiuae/falcon-7b",
model_name = "tiiuae/falcon-7b",
device_map = 'auto',
# uncomment this if using CUDA to reduce memory usage
model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
embed_model = LangchainEmbedding(
    HuggingFaceBgeEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
)

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size = 1024,
    llm = llm,
    embed_model = embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context= service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query = "What is the difference between attention and self-attention?"
response = query_engine.query(query)
print(response)